# Exercise 10: 

In this week's exercise, we will be working from the class demo. To use the live features, you'll need to get a Twitter account approved through the Developer Portal (Links to an external site.) as reviewed in class. If you can't get approval, you can use the alternate method of importing your data via TAGS (Links to an external site.)

Your application should:

- Import or collect tweets using a keyword or hashtag. Watch your rate limiting!
- Use Pandas to sort and analyze your tweets.
- Calculate user coordinates, where feasible, for the set of collected tweets. Again, watch your rate limiting!
- Map the tweets using the calculated locations.
- Use at least one data visualization from a previous exercise, demo, or the course text to analyze a different element of your tweets.

As a bonus challenge, try to map some other quality of your data. For instance, you might only map the locations where a certain word or hashtag is used, or try to visualize the quantity of tweets that share a location using a larger radius dot.


## Stage One: Import or collect tweets using a keyword or hashtag. Watch your rate limiting!


In [ ]:
import configparser
CONFIG = configparser.ConfigParser()
CONFIG.read('credentials.ini')

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(CONFIG['DEFAULT']['consumer_key'], CONFIG['DEFAULT']['consumer_secret'])
auth.set_access_token(CONFIG['DEFAULT']['access_token'], CONFIG['DEFAULT']['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

## Stage Two:  Use Pandas to sort and analyze your tweets.

In [ ]:
import pandas as pd

search_words = "#squidgame"
date_since = "2021-10-01"
tweets = tweepy.Cursor(api.search_tweets,
              q=search_words, lang="en").items(300)

tweets_sorted = [[tweet.user.screen_name, tweet.geo, tweet.user.location, tweet.text] for tweet in tweets]

## Stage Three: Calculate user coordinates, where feasible, for the set of collected tweets. Again, watch your rate limiting! 

In [ ]:
tdf = pd.DataFrame(data=tweets_sorted, columns=['user', 'coordinates','location', 'tweet'])
print(tdf)

In [ ]:
locs = tdf['location'].value_counts()
print(locs)

## Stage Four:  Map the tweets using the calculated locations.

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent='twitter-analysis-client')
limited = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def find_location(row):
    place = row['location']
    location = limited(place)
    
    if location != None:
        return location.latitude, location.longitude
    else:
        return "Not Found", "Not Found"

tdf[['latitude','longitude']] = tdf.apply(find_location, axis="columns", result_type="expand")
tdf

## Stage Five: Use at least one data visualization from a previous exercise, demo, or the course text to analyze a different element of your tweets.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 20})
plt.rcParams['figure.figsize'] = (20, 10)

## Bonus Challenge: Try to map some other quality of your data. For instance, you might only map the locations where a certain word or hashtag is used, or try to visualize the quantity of tweets that share a location using a larger radius dot.



In [ ]:
import cartopy.crs as ccrs
from matplotlib.patches import Circle

lats = tdf['latitude'].dropna()
longs = tdf['longitude'].dropna()

ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
    
for i in range(0, len(lats)):
    ax.plot(longs[i], lats[i], 'ro', transform=ccrs.PlateCarree())
                                                                                              
ax.coastlines(resolution='50m')
ax.set_global()

plt.show()